In [ ]:
pip install arabert

## Arabert

This code will fine-tune the BERT model on the training data, and then test the model on the test data, printing the accuracy of the predictions. In this example, the dataframe is expected to have two columns, one for the text (named "text") and one for the labels (named "hard_label"). The labels are expected to be in boolean format, 0 or 1.
You may need to import tokenizer from transformers library before using it.
Please also note that this is a simplified example, you may want to consider adding other metrics like F1-score, precision, recall and so on, or using K-fold cross validation to get a more robust evaluation of your model.

In [2]:
import pandas as pd
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import BertTokenizer
import torch

In [1]:
path = r'G:/Il mio Drive/SemEval_Task11/LearningWithDisagreements/Data/data_practicephase_cleardev/data_practicephase_cleardev/'

In [3]:
from tqdm import tqdm

In [4]:
# datasets paths
armis_train = path + 'ArMIS_dataset/ArMIS_train.json'
armis_dev = path + 'ArMIS_dataset/ArMIS_dev.json'


In [5]:
# import and concat train datasets
df = pd.read_json(armis_train, orient='index')
df = df[['text', 'hard_label']]

In [6]:
df_dev = pd.read_json(armis_dev, orient='index')
df_dev = df_dev[['text', 'hard_label']]

In [7]:
df

,text,hard_label
1,النسويه يعني نصير رجل قولتك وبعدين اذا الوحده ...,0
2,هراء النسويات عمان والمتستر بتضخيم حالات العنف...,1
3,بسبب انتشار الفكر النسوي القد ودعوة النسويات ا...,1
4,وقال تعالى الن الل احا الل تأملي هذة الأيه,0
5,يعني السعوديات صايعات يتنقلو واحد للثاني,0
...,...,...
653,رأيج محترم جدا ومنطقي وطرحج معقول اغلب اكدر اك...,1
654,تتع مئات الآلاف الفتيات والنساء يوميا جميع أنح...,0
655,وغيرها الداخل والخارج سافرات ومتعديات حدود الد...,1
656,والله ياصديقي انا اشفق عليها يسعفها عقلها ولم ...,1


In [39]:
# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
print(torch. __version__)
#python -m pip install torch==1.7.0 -f https://download.pytorch.org/whl/torch_stable.html

In [9]:
from arabert import ArabertPreprocessor
from arabert.aragpt2.grover.modeling_gpt2 import GPT2LMHeadModel

In [36]:
# select a model from the huggingface modelhub https://huggingface.co/models?language=ar
model_name = 'aubmindlab/bert-base-arabertv02-twitter' # we are going to use the twitter AraBERT since it has emojis and dialects


In [11]:
#Create and apply preprocessing using the AraBERT processor
arabic_prep = ArabertPreprocessor(model_name)

In [12]:
df['text'] = df['text'].apply(lambda x: arabic_prep.preprocess(x))
df

,text,hard_label
1,النسويه يعني نصير رجل قولتك وبعدين اذا الوحده ...,0
2,هراء النسويات عمان والمتستر بتضخيم حالات العنف...,1
3,بسبب انتشار الفكر النسوي القد ودعوة النسويات ا...,1
4,وقال تعالى الن الل احا الل تأملي هذة الأيه,0
5,يعني السعوديات صايعات يتنقلو واحد للثاني,0
...,...,...
653,رأيج محترم جدا ومنطقي وطرحج معقول اغلب اكدر اك...,1
654,تتع مئات الآلاف الفتيات والنساء يوميا جميع أنح...,0
655,وغيرها الداخل والخارج سافرات ومتعديات حدود الد...,1
656,والله ياصديقي انا اشفق عليها يسعفها عقلها ولم ...,1


In [34]:
from transformers import (AutoConfig, AutoModelForSequenceClassification,
                          AutoTokenizer, BertTokenizer, Trainer,
                          TrainingArguments)

In [37]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [17]:
# Helper function for formatting inputs
def format_inputs(text, labels):
    input_ids = torch.tensor([tokenizer.encode(text, add_special_tokens=True)])
    attention_mask = torch.tensor([[1]*len(input_ids[0])])
    labels = torch.tensor([labels])
    return input_ids, attention_mask, labels

In [18]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, return_dict=True, num_labels=2)

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02-twitter were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmi

In [19]:
# Define the optimizer and schedule (linear warmup and decay)
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)

c:\Users\giuli\anaconda3\envs\Semeval23\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [20]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(64000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
# Fine-tune the model
for epoch in tqdm(range(20)):
    model.train()
    train_loss = 0
    for i, row in df.iterrows():
        input_ids, attention_mask, labels = format_inputs(row['text'], row['hard_label'])
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {train_loss/len(df)}')

In [61]:
torch.save(model, './model/arabert/fine_tuned_arabert.pth')

In [ ]:
cd ../

In [5]:
model=torch.load('./model/arabert/fine_tuned_arabert.pth')

In [ ]:
# Test the model
model.eval()
test_loss = 0
for i, row in df_tot.iterrows():
    input_ids, attention_mask, labels = format_inputs(row['text'], row['hard_label'])
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    labels = labels.to(device)

    outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs[0]
    test_loss += loss

In [62]:
# Helper function for getting predictions
def get_predictions(model, dataframe):
    predictions = []
    model.eval()
    for i, row in dataframe.iterrows():
        input_ids, attention_mask, _ = format_inputs(row['text'], row['hard_label'])
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs[0]
        pred = logits.argmax(dim=1).item()
        predictions.append(pred)
    return predictions

In [ ]:
# SE SU CPU
model = model.to(device)

In [63]:
# Get predictions for the test set
test_predictions = get_predictions(model, df_dev)

In [64]:
# Compare predictions to true labels
df_dev['predictions'] = test_predictions
correct_predictions = df_dev[df_dev['hard_label'] == df_dev['predictions']]
accuracy = len(correct_predictions) / len(df_dev)
print(f'Accuracy: {accuracy}')

Accuracy: 0.7659574468085106


### Integrated Gradient

This code will use the integrated gradients method to calculate the attribution of each token in the input text to the final prediction of the model. The result is a list of values for each token, showing how much each token contributes to the final prediction.
You can also use other attribution methods such as ShapleyValue, Saliency or Deconvolution that are also available in captum library.
Please also note that this is a simplified example, you may want to consider adding other modifications such as running the explanation for multiple inputs to get a better idea of the model's behavior.

In [6]:
from captum.attr import IntegratedGradients

https://github.com/pytorch/captum/issues/150

In [29]:
import torch
import torch.nn as nn

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, BertConfig

from captum.attr import IntegratedGradients
from captum.attr import InterpretableEmbeddingBase, TokenReferenceBase
from captum.attr import visualization
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer


# We need to split forward pass into two part: 
# 1) embeddings computation
# 2) classification

def compute_bert_outputs(model_bert, embedding_output, attention_mask=None, head_mask=None):
    if attention_mask is None:
        attention_mask = torch.ones(embedding_output.shape[0], embedding_output.shape[1]).to(embedding_output)

    extended_attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)

    extended_attention_mask = extended_attention_mask.to(dtype=next(model_bert.parameters()).dtype) # fp16 compatibility
    extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0

    if head_mask is not None:
        if head_mask.dim() == 1:
            head_mask = head_mask.unsqueeze(0).unsqueeze(0).unsqueeze(-1).unsqueeze(-1)
            head_mask = head_mask.expand(model_bert.config.num_hidden_layers, -1, -1, -1, -1)
        elif head_mask.dim() == 2:
            head_mask = head_mask.unsqueeze(1).unsqueeze(-1).unsqueeze(-1)  # We can specify head_mask for each layer
        head_mask = head_mask.to(dtype=next(model_bert.parameters()).dtype) # switch to fload if need + fp16 compatibility
    else:
        head_mask = [None] * model_bert.config.num_hidden_layers

    encoder_outputs = model_bert.encoder(embedding_output,
                                         extended_attention_mask,
                                         head_mask=head_mask)
    sequence_output = encoder_outputs[0]
    pooled_output = model_bert.pooler(sequence_output)
    outputs = (sequence_output, pooled_output,) + encoder_outputs[1:]  # add hidden_states and attentions if they are here
    return outputs  # sequence_output, pooled_output, (hidden_states), (attentions)    

In [30]:
class BertModelWrapper(nn.Module):
    
    def __init__(self, model):
        super(BertModelWrapper, self).__init__()
        self.model = model
        
    def forward(self, embeddings):        
        outputs = compute_bert_outputs(self.model.bert, embeddings)
        pooled_output = outputs[1]
        pooled_output = self.model.dropout(pooled_output)
        logits = self.model.classifier(pooled_output)
        return torch.softmax(logits, dim=1)[:, 1].unsqueeze(1)

In [31]:
bert_model_wrapper = BertModelWrapper(model)
ig = IntegratedGradients(bert_model_wrapper)

In [9]:
# accumalate couple samples in this array for visualization purposes
vis_data_records_ig = []

https://github.com/pytorch/captum/blob/master/captum/attr/_utils/visualization.py

In [19]:
def interpret_sentence(model_wrapper, sentence, label=1):

    model_wrapper.eval()
    model_wrapper.zero_grad()
    
    print(torch.tensor([tokenizer.encode(sentence, add_special_tokens=True)]))
    input_ids = torch.tensor([tokenizer.encode(sentence, add_special_tokens=True)]).to(device)
    
    input_embedding = model_wrapper.model.bert.embeddings(input_ids)
    
    # predict
    pred = model_wrapper(input_embedding).item()
    pred_ind = round(pred)

    # compute attributions and approximation delta using integrated gradients
    attributions_ig, delta = ig.attribute(input_embedding, n_steps=100, return_convergence_delta=True)

    print('pred: ', pred_ind, '(', '%.2f' % pred, ')', ', delta: ', abs(delta))
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0].cpu().data.numpy().tolist())    
    
    #add_attributions_to_visualizer(attributions_ig, tokens, pred, pred_ind, label, delta, vis_data_records_ig)
    return attributions_ig, tokens, pred, pred_ind, label, delta, vis_data_records_ig
    
    
def add_attributions_to_visualizer(attributions, tokens, pred, pred_ind, label, delta, vis_data_records):
    attributions = attributions.sum(dim=2).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    attributions = attributions.detach().cpu().data.numpy()
    
    # storing couple samples in an array for visualization purposes
    vis_data_records.append(visualization.VisualizationDataRecord(
                            attributions,
                            pred,
                            pred_ind,
                            label,
                            "hard_label",
                            attributions.sum(),       
                            tokens[:len(attributions)],
                            delta))    


In [77]:
df_dev[1:2]

,text,hard_label,predictions
2,النسويات الفسويات,1,1


In [21]:
attributions_ig, tokens, pred, pred_ind, label, delta, vis_data_records_ig = interpret_sentence(bert_model_wrapper, df_dev.loc[1,'text'], label=1)
add_attributions_to_visualizer( attributions_ig, tokens, pred, pred_ind, label, delta, vis_data_records_ig)

tensor([[    2,   487, 12167, 18905, 19484, 38055,  5014,  7461,  8987,   870,
          1962,  1523,  7849,  7632, 27738,  1316,  5805, 19345,  5645,  3530,
         10918, 34082,     3]])
pred:  0 ( 0.24 ) , delta:  tensor([0.0044], device='cuda:0', dtype=torch.float64)


In [22]:
visualization.visualize_text(vis_data_records_ig)

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,0 (0.24),hard_label,0.65,[CLS] الي يقرأ اسئلة المعلمات يستغرب انهم يعملون معنا نفس الوزارة الكثير القيادات النسائية متسل ##طات فعلا عندهم سوء فهم وتطبيق للأنظمة [SEP]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,0 (0.24),hard_label,0.65,[CLS] الي يقرأ اسئلة المعلمات يستغرب انهم يعملون معنا نفس الوزارة الكثير القيادات النسائية متسل ##طات فعلا عندهم سوء فهم وتطبيق للأنظمة [SEP]


In [10]:
def interpret_sentence(model_wrapper, sentence, label=1):

    model_wrapper.eval()
    model_wrapper.zero_grad()
    
    input_ids = torch.tensor([tokenizer.encode(sentence, add_special_tokens=True)]).to(device)
    
    input_embedding = model_wrapper.model.bert.embeddings(input_ids)
    

    # compute attributions and approximation delta using integrated gradients
    attributions_ig, _ = ig.attribute(input_embedding, n_steps=100, return_convergence_delta=True)
    del model_wrapper
    gc.collect()
    torch.cuda.empty_cache()

    #print('pred: ', pred_ind, '(', '%.2f' % pred, ')', ', delta: ', abs(delta))
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0].cpu().data.numpy().tolist())    
    
    #add_attributions_to_visualizer(attributions_ig, tokens, pred, pred_ind, label, delta, vis_data_records_ig)
    return attributions_ig, tokens

In [24]:
df_dev

,text,hard_label
1,الي يقرأ اسئلة المعلمات يستغرب انهم يعملون معن...,0
2,النسويات الفسويات,1
3,سؤال للـ بسيط اتوقع تكلمنا التويتر دورات ومحاض...,1
4,وبكذا أوكد استنتاجي بإن النسويات قسمين القسم ا...,1
5,انت قلتي قلتي النسويه امرأة,0
...,...,...
137,الرياضات القتالية حيلة نساء لمواجهة العنف المرأة,0
138,احب النوع الفسويات لانها تجيب لحالها المسبه تنسحب,1
139,اول شوف اخواتك وبعدين تكلم مكان صايعات ... عند...,1
140,ليييش ليش يحطون شيء بالنسويه خير مالقوا سبب قا...,0


In [11]:
from tqdm import tqdm
import gc

In [30]:
df_dev['attention']=0
df_dev['tokens']=''
for index, row in tqdm(df_dev.iterrows()):
  attributions_ig, tokens = interpret_sentence(bert_model_wrapper, sentence=df_dev.loc[index, 'text'], label=0)
  attributions_ig = attributions_ig.sum(dim=2).squeeze(0)
  attributions_ig = attributions_ig / torch.norm(attributions_ig)
  attributions_ig = attributions_ig.detach().cpu().data.numpy()
  df_dev.loc[index, 'attention'] = str(attributions_ig)
  df_dev.loc[index, 'tokens'] = str(tokens)

141it [00:43,  3.22it/s]


In [31]:
df_dev

,text,hard_label,attention,tokens
1,الي يقرأ اسئلة المعلمات يستغرب انهم يعملون معن...,0,[ 0.1534795 -0.030359 -0.09889226 -0.190681...,"['[CLS]', 'الي', 'يقرأ', 'اسئلة', 'المعلمات', ..."
2,النسويات الفسويات,1,[ 0.26387608 0.04718343 0.01067558 0.654362...,"['[CLS]', 'النسوي', '##ات', 'الفس', '##ويات', ..."
3,سؤال للـ بسيط اتوقع تكلمنا التويتر دورات ومحاض...,1,[ 9.69040651e-01 -4.62940413e-02 7.14545812e-...,"['[CLS]', 'سؤال', '[UNK]', 'بسيط', 'اتوقع', 'ت..."
4,وبكذا أوكد استنتاجي بإن النسويات قسمين القسم ا...,1,[ 0.07688863 -0.10141717 0.11480371 -0.484018...,"['[CLS]', 'وبك', '##ذا', 'أوك', '##د', 'استنتا..."
5,انت قلتي قلتي النسويه امرأة,0,[ 0.21483734 0.37937387 0.29936994 -0.129498...,"['[CLS]', 'انت', 'قلت', '##ي', 'قلت', '##ي', '..."
...,...,...,...,...
137,الرياضات القتالية حيلة نساء لمواجهة العنف المرأة,0,[ 0.94781497 0.04148038 0.02542426 0.011141...,"['[CLS]', 'الرياضات', 'القتالية', 'حيلة', 'نسا..."
138,احب النوع الفسويات لانها تجيب لحالها المسبه تنسحب,1,[-0.05101407 -0.13024441 -0.10013564 0.634415...,"['[CLS]', 'احب', 'النوع', 'الفس', '##ويات', 'ل..."
139,اول شوف اخواتك وبعدين تكلم مكان صايعات ... عند...,1,[-0.17444202 0.00055693 0.26633095 -0.397172...,"['[CLS]', 'اول', 'شوف', 'اخواتك', 'وبعدين', 'ت..."
140,ليييش ليش يحطون شيء بالنسويه خير مالقوا سبب قا...,0,[ 0.23175431 -0.14728426 -0.31090029 -0.038534...,"['[CLS]', 'لي', '##يي', '##ش', 'ليش', 'يحطو', ..."


In [32]:
df_dev.to_csv('./IG/arabic.csv', sep='\t', index=False)

## IG on Test

In [18]:
path = r'G:\Il mio Drive\SemEval_Task11\LearningWithDisagreements\Data\data_evaluation'

In [19]:
# datasets paths
armis_test = path + '\ArMIS_test.json'


In [26]:
df_test = pd.read_json(armis_test, orient='index')

In [27]:
df_test

,text,annotation task,number of annotations,annotations,annotators,lang,hard_label,soft_label,split,other_info
1,اللسان السليط الص راخ الجراءة العنهجية والهمجي...,misogyny and sexism detection in Arabic tweets,3,,"Ann1,Ann2,Ann3",ar,NaN,"{'0': None, '1': None}",test,"{'Ann1': 'Moderate_Female', 'Ann2': 'Liberal_F..."
2,قناة الطهر أفضل قناة لمن أراد يحافظ أولاده تخد...,misogyny and sexism detection in Arabic tweets,3,,"Ann1,Ann2,Ann3",ar,NaN,"{'0': None, '1': None}",test,"{'Ann1': 'Moderate_Female', 'Ann2': 'Liberal_F..."
3,عنف يقال سيبويه أحرقت زوجته كتبه لأنه كان يشتغ...,misogyny and sexism detection in Arabic tweets,3,,"Ann1,Ann2,Ann3",ar,NaN,"{'0': None, '1': None}",test,"{'Ann1': 'Moderate_Female', 'Ann2': 'Liberal_F..."
4,ليش لازم شيء نسويه عشان شخص لازم ترى عادي اقوم...,misogyny and sexism detection in Arabic tweets,3,,"Ann1,Ann2,Ann3",ar,NaN,"{'0': None, '1': None}",test,"{'Ann1': 'Moderate_Female', 'Ann2': 'Liberal_F..."
5,اساس قارنتوا طاعه الزوج بطاعة الرب عجزت افهم ا...,misogyny and sexism detection in Arabic tweets,3,,"Ann1,Ann2,Ann3",ar,NaN,"{'0': None, '1': None}",test,"{'Ann1': 'Moderate_Female', 'Ann2': 'Liberal_F..."
...,...,...,...,...,...,...,...,...,...,...
141,تخيل مطوع قال ناقصات عقل دين,misogyny and sexism detection in Arabic tweets,3,,"Ann1,Ann2,Ann3",ar,NaN,"{'0': None, '1': None}",test,"{'Ann1': 'Moderate_Female', 'Ann2': 'Liberal_F..."
142,تخيلي اقول الله ياخذ النساء اذا وحدة زعلت ردت ...,misogyny and sexism detection in Arabic tweets,3,,"Ann1,Ann2,Ann3",ar,NaN,"{'0': None, '1': None}",test,"{'Ann1': 'Moderate_Female', 'Ann2': 'Liberal_F..."
143,الله يرحمه يغفرله يجعل مثواه الجنه يارب العالم...,misogyny and sexism detection in Arabic tweets,3,,"Ann1,Ann2,Ann3",ar,NaN,"{'0': None, '1': None}",test,"{'Ann1': 'Moderate_Female', 'Ann2': 'Liberal_F..."
144,مافي الزمن النبي قال زمن الأزمنة يكون النساء أ...,misogyny and sexism detection in Arabic tweets,3,,"Ann1,Ann2,Ann3",ar,NaN,"{'0': None, '1': None}",test,"{'Ann1': 'Moderate_Female', 'Ann2': 'Liberal_F..."


In [40]:
df_test['attention']=0
df_test['tokens']=''
for index, row in tqdm(df_test.iterrows()):
  attributions_ig, tokens = interpret_sentence(bert_model_wrapper, sentence=df_test.loc[index, 'text'], label=0)
  attributions_ig = attributions_ig.sum(dim=2).squeeze(0)
  attributions_ig = attributions_ig / torch.norm(attributions_ig)
  attributions_ig = attributions_ig.detach().cpu().data.numpy()
  df_test.loc[index, 'attention'] = str(attributions_ig)
  df_test.loc[index, 'tokens'] = str(tokens)

145it [00:52,  2.78it/s]


In [42]:
df_test.to_csv('./IG/arabic_test.csv', sep='\t', index=False)